In [1]:
from flask import Flask, render_template, request, redirect, url_for
import pymysql

app = Flask(__name__)

mydb = pymysql.connect(host="localhost",user="root",password="cic2020",database="mysteamdb") # 데스크탑 1234, 노트북 cic2020
mycursor = mydb.cursor()

@app.route('/')     # 구매한 게임 리스트 (사전 순 정렬)
def game_list():
    mycursor.execute(''' 
    SELECT pi.purchase_date, gi.game_name, ge.genre_name, pi.purchase_price, di.developer_name, ri.rating, pi.game_id
    FROM purchase_info pi
    LEFT JOIN game_info gi ON pi.game_id = gi.game_id
    LEFT JOIN genres_info ge ON gi.genre_id = ge.genre_id
    LEFT JOIN developers_info di ON gi.developer_id = di.developer_id
    LEFT JOIN reviews_info ri ON gi.game_id = ri.game_id
    order by gi.game_name
    ''' )
    re = mycursor.fetchall()
    return render_template("game_list.html", item_list = re)

@app.route('/add_game', methods=['GET', 'POST'])    # 구매 게임 추가
def add_game():
    if request.method == 'POST':
        game_name = request.form['game_name']
        game_genre = request.form['game_genre']
        purchase_price = request.form['purchase_price']
        game_price = request.form['game_price']
        developer_name = request.form['developer_name']
        purchase_date = request.form['purchase_date']
        
        # 장르 테이블 업데이트(중복 제외)
        update_genre = (game_genre, game_genre)
        mycursor.execute('''
        insert into genres_info(genre_name)
        select %s
        from dual 
        where not exists
            (select * from genres_info 
            where genre_name = %s);
        ''', update_genre)
        
        # 제작사 테이블 업데이트(중복 제외)
        update_developer = (developer_name, developer_name)
        mycursor.execute('''
        insert into developers_info(developer_name)
        select %s
        from dual 
        where not exists
            (select * from developers_info
            where developer_name = %s);
        ''', update_developer)
            
        # 게임 정보 테이블 업데이트
        update_game = (game_name, game_price, game_genre, developer_name)
        mycursor.execute('''
        insert into game_info(game_name, genre_id, price, developer_id)
        select %s, genre_id, %s, developer_id
        from genres_info, developers_info
        where genres_info.genre_name = %s and developers_info.developer_name = %s
        ''', update_game)
        
        # 구매 정보 테이블 업데이트
        update_purchase = (purchase_date, purchase_price, game_name)
        mycursor.execute('''
        insert into purchase_info(game_id, purchase_date, purchase_price)
        select game_id, %s, %s
        from game_info
        where game_info.game_name = %s
        ''', update_purchase)
        
        mydb.commit()
            
    return render_template("add_game.html")
 
@app.route('/game_info/<game_name>/<game_id>')    # 선택한 게임에 대한 정보(+ 삭제 및 평가)
def game_info(game_name, game_id):
    mycursor.execute('''
    select gi.game_name, ge.genre_name, gi.price, di.developer_name, gi.game_id
    from game_info gi
    join genres_info ge on gi.genre_id = ge.genre_id
    join developers_info di on gi.developer_id = di.developer_id
    where gi.game_id = %s
    ''', game_id)
    
    re = mycursor.fetchall()
    
    return render_template("game_info.html", game_name = game_name, item_list = re, game_id = game_id)

@app.route('/delete_game/<game_id>')  # 게임 삭제(장르랑 제작사는 다시 사용할 수 있으니 굳이 삭제 안함)
def delete_game(game_id):
    # purchase_info 데이터 삭제
    mycursor.execute('''
    delete from purchase_info 
    where game_id = %s
    ''', game_id)
    
    # reviews_info 데이터 삭제
    mycursor.execute('''
    delete from reviews_info 
    where game_id = %s
    ''', game_id)
    
    # game_info 데이터 삭제
    mycursor.execute('''
    delete from game_info
    where game_id = %s
    ''', game_id)
    
    mydb.commit()
    
    return redirect(url_for('game_list'))

@app.route('/game_review/<game_name>/<game_id>', methods=['GET', 'POST']) # 평점 및 후기 
def game_review(game_name, game_id):
    if request.method == 'POST':
        rating = request.form['rating']
        comment = request.form['comment']
        
        review_update = (rating, comment, game_id)
        
        mycursor.execute('''
        select ri.review_id from reviews_info ri
        join game_info gi on gi.game_id = ri.game_id
        where gi.game_id = %s
        ''', game_id)
        
        is_review = mycursor.fetchall()
        
        if not is_review:
            mycursor.execute('''
            insert into reviews_info(rating, comment, game_id)
            values (%s, %s, %s)
            ''', review_update)
        else:
            mycursor.execute('''
            update reviews_info
                set rating = %s,
                    comment = %s
            where game_id = %s
            ''', review_update)
            
        mydb.commit()
        return redirect(url_for('game_review', game_name = game_name ,game_id = game_id))
    
    else:
        mycursor.execute('''
        select ri.rating, ri.comment
        from game_info gi
        left join reviews_info ri on gi.game_id = ri.game_id
        where gi.game_id = %s
        ''',game_id)
        
        review_data = mycursor.fetchone()
        return render_template("game_review.html", game_name = game_name, review_data = review_data)

@app.route('/game_date/<purchase_date>')    # 선택한 날짜에 구매한 게임 리스트
def game_date(purchase_date):
    mycursor.execute('''
    select gi.game_name, ge.genre_name, pi.purchase_price, di.developer_name
    from game_info gi
    join genres_info ge on gi.genre_id = ge.genre_id
    join purchase_info pi on gi.game_id = pi.game_id
    join developers_info di on gi.developer_id = di.developer_id
    where pi.purchase_date = %s
    order by gi.game_name
    ''', purchase_date)
    
    re = mycursor.fetchall()
    return render_template("game_date.html", template_name = purchase_date, item_list = re)

@app.route('/game_genre/<genre_name>')      # 선택한 장르 게임 구매 리스트
def game_genre(genre_name):
    mycursor.execute('''
    select gi.game_name, gi.price, di.developer_name
    from game_info gi
    join genres_info ge on gi.genre_id = ge.genre_id
    join developers_info di on gi.developer_id = di.developer_id
    where ge.genre_name = %s
    order by gi.game_name
    ''', genre_name)
    
    re = mycursor.fetchall()
    return render_template("game_genre.html", template_name = genre_name, item_list = re)

@app.route('/game_developer/<developer_name>')  # 선택한 제작사 게임 구매 리스트
def game_developer(developer_name):
    mycursor.execute('''
    select gi.game_name, ge.genre_name, gi.price
    from game_info gi
    join genres_info ge on gi.genre_id = ge.genre_id
    join developers_info di on gi.developer_id = di.developer_id
    where di.developer_name = %s
    order by gi.game_name
    ''', developer_name)
    
    re = mycursor.fetchall()
    return render_template("game_developer.html", template_name = developer_name, item_list = re)

@app.route('/sum_amount')   # 사용 금액에 대한 정보
def sum_amount():
    mycursor.execute('''
    select sum(pi.purchase_price)d
	, sum(gi.price)
    , (sum(gi.price) - sum(pi.purchase_price))
    , concat(round(((sum(gi.price) - sum(pi.purchase_price)) / sum(gi.price)) * 100, 2), "%")
    from purchase_info pi, game_info gi
    where pi.game_id = gi.game_id;
    ''')
    
    re = mycursor.fetchall()
    return render_template("sum_amount.html", item_list = re)
    
if __name__ == '__main__':
    
    app.run()
    


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Jun/2024 14:04:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2024 14:04:48] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Jun/2024 14:04:52] "GET /add_game HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2024 14:05:37] "POST /add_game HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2024 14:05:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2024 14:05:43] "GET /sum_amount HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2024 14:05:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2024 14:05:51] "GET /game_info/포켓몬스터/6 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2024 14:05:53] "GET /game_review/포켓몬스터/6 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2024 14:05:58] "POST /game_review/포켓몬스터/6 HTTP/1.1" 302 -
127.0.0.1 - - [10/Jun/2024 14:05:58] "GET /game_review/포켓몬스터/6 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2024 14:05:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2024 14:06:05] "GET /game_developer/일본 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2024 14:06: